#### このノート
- data: X_ori55_ad, ori_y5
- CNF式を再現してみた。
- 1の総数は指定しなかった方法と、した方法
- 単数出力、複数出力



#### 教訓
**変数aに変数bを代入し、その後変数bの値を変える場合、変数aは代入時の値と変わってしまう**

対策
- copy.copy('代入したい値')
- copy.deepcopy('代入したい値')
- '代入したい値'.copy()
- '代入したい値'[:]


**ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()**

対策
- とりあえず`a.all()`を当てはめてみる

https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=7163203

In [280]:
import copy
import itertools
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, f_regression
import random
from pyqubo import Array, OneHotEncInteger, solve_qubo
import scipy.stats

In [281]:
#標準偏差が0である列をなくす
def drop_str0_X(X):
    for each_col in X.columns:
        if X[each_col].std()==0.0:
            X = X.drop(each_col, axis=1)
    return X

In [283]:
X_ori55 = pd.read_csv("../input/SNP_df55.csv", sep=',', index_col=0)
X_ori55_ad = drop_str0_X(X_ori55)

num_samples = X_ori55_ad.shape[0]

In [284]:
ori_y5 = pd.read_csv('../input/y5.csv', sep=',', index_col=0)['y']

y1_sum = sum(ori_y5)
#y1_sum #3

In [24]:
def featrure_InnerProduct_sel_mul(X, y, sel_col_num):
    inner_product_list = [np.dot(X[each_col], y) for each_col in X.columns] #内積のリスト
    sort_inner_product_list = sorted(inner_product_list ,reverse=True)
    
    select_cols = []
    for i in range(sel_col_num):
        iindex = inner_product_list.index(sort_inner_product_list[i])
        select_cols.append(X.columns[iindex])
    
    else_cols = list(X.columns)
    for col in select_cols:
        else_cols.remove(col)
    
    return select_cols, else_cols

In [25]:
selected_col_num2 = 2
#select_cols, else_cols = featrure_InnerProduct_sel_mul(X_ori55_ad, ori_y5, selected_col_num2)

## SATのシステムをオリジナルで書いてみる
- 単数出力、1の総数未指定
- 複数出力、1の総数未指定
- 単数出力、1の総数指定
- 複数出力、1の総数指定

In [156]:
infer_y0 = pd.Series([0, 1, 1, 0, 1])#とする
infer_y0

0    0
1    1
2    1
3    0
4    1
dtype: int64

In [179]:
#type(infer_y0[0]) =='numpy.int64'  #難しいな

False

In [157]:
dot_list_ori = [np.dot(X_ori55_ad[i], ori_y5) - np.dot(X_ori55_ad[j], ori_y5) for i in select_cols for j in else_cols]
dot_list_ori #全て内積は0以上となっている（エラーは起きていない）

[1, 1, 1, 0, 0, 0]

In [ ]:
#合っているか確認
def check_decide_y_fuc(X, ori_y, selected_col_num, deci_y):
    select_cols, else_cols = featrure_InnerProduct_sel_mul(X, ori_y, selected_col_num)
    
    dot_list_confirm = [np.dot(X[i], deci_y) - np.dot(X[j], deci_y) for i in select_cols for j in else_cols]
    if len([ll for ll in dot_list_confirm if ll <0]) != 0:
        print('{} is wrong'.format(deci_y.values.tolist()))

In [213]:
check_decide_y_fuc(X_ori55_ad, ori_y5, selected_col_num2, ori_y5)

In [170]:
if len([ll for ll in dot_list_ori if ll <0]) == 0:
    print('there is no problem')

there is no problem


In [181]:
#合っているか確認(複数出力)
def check_decide_y_fuc_mul(X, ori_y, selected_col_num, deci_y_list):
    select_cols, else_cols = featrure_InnerProduct_sel_mul(X, ori_y, selected_col_num)

    for m in deci_y_list:
        dot_list_confirm = [np.dot(X[i], m) - np.dot(X[j], m) for i in select_cols for j in else_cols]
        if len([ll for ll in dot_list_confirm if ll <0]) == 0:
            continue
        else:
            print('{} is wrong'.format(m.values.tolist()))

### 単数出力、1の総数未指定

In [232]:
# 出力一つ
def decide_y(X, ori_y, infer_y, selected_col_num):
    select_cols, else_cols = featrure_InnerProduct_sel_mul(X, ori_y, selected_col_num)
    deci_y = 0
    dot_list = [np.dot(X[i], infer_y) - np.dot(X[j], infer_y) for i in select_cols for j in else_cols]
    if len([ll for ll in dot_list if ll <0]) == 0:
        print(dot_list)#list
        print(infer_y)#series
        print('-----------------')
        deci_y = infer_y
        return deci_y
    else:
        for kk in reversed(range(num_samples)):
            for k in reversed(range(kk+1)):
                infer_y[k] = 1 - infer_y[k]
                dot_list = [np.dot(X[i], infer_y) - np.dot(X[j], infer_y) for i in select_cols for j in else_cols]
                if len([ll for ll in dot_list if ll <0]) == 0:
                    print(dot_list)#list
                    print(infer_y)#series
                    print('-----------------')
                    deci_y = infer_y
                    return deci_y
                else:
                    infer_y[k] = 1 - infer_y[k]
            infer_y[kk] = 1 - infer_y[kk]
    return deci_y #series

In [235]:
infer_y0 = pd.Series([0, 1, 1, 0, 1])
#deci_y0 = decide_y(X_ori55_ad, ori_y5, infer_y0, selected_col_num2)#elseへ進んだ
deci_y0 = decide_y(X_ori55_ad, ori_y5, infer_y0, selected_col_num2)
deci_y0

[0, 0, 2, 0, 0, 2]
0    0
1    1
2    1
3    1
4    0
dtype: int64
-----------------


0    0
1    1
2    1
3    1
4    0
dtype: int64

In [237]:
check_decide_y_fuc(X_ori55_ad, ori_y5, selected_col_num2, deci_y0)

In [238]:
dot_list_confirm0 = [np.dot(X_ori55_ad[i], deci_y0) - np.dot(X_ori55_ad[j], deci_y0) for i in select_cols for j in else_cols]
dot_list_confirm0

[0, 0, 2, 0, 0, 2]

In [257]:
# 出力複数
def decide_y_mul(X, ori_y, infer_y, selected_col_num):
    select_cols, else_cols = featrure_InnerProduct_sel_mul(X, ori_y, selected_col_num)
    deci_y_list = []
    dot_list = [np.dot(X[i], infer_y) - np.dot(X[j], infer_y) for i in select_cols for j in else_cols]
    
    if len([ll for ll in dot_list if ll <0]) == 0:
        print(dot_list)#list
        deci_y_list.append(copy.deepcopy(infer_y))
        print(infer_y)#series
        print('-----------------')
    for kk in reversed(range(num_samples)):
        for k in reversed(range(kk+1)):
            infer_y[k] = 1 - infer_y[k]
            dot_list = [np.dot(X[i], infer_y) - np.dot(X[j], infer_y) for i in select_cols for j in else_cols]
            if len([ll for ll in dot_list if ll <0]) == 0:
                if len(deci_y_list) == 0:
                    print(dot_list)#list
                    print(infer_y)#series
                    print('-----------------')
                    deci_y_list.append(copy.deepcopy(infer_y))
                elif (deci_y_list[-1] == infer_y).all() == False:
                    print(dot_list)#list
                    print(infer_y)#series
                    print('-----------------')
                    deci_y_list.append(copy.deepcopy(infer_y))
            infer_y[k] = 1 - infer_y[k]
        infer_y[kk] = 1 - infer_y[kk]
    
    return deci_y_list

In [258]:
infer_y0 = pd.Series([0, 1, 1, 0, 1])
deci_y1 = decide_y_mul(X_ori55_ad, ori_y5, infer_y0, selected_col_num2)
deci_y1
#[[1, 1, 0, 1, 0]]

[0, 0, 2, 0, 0, 2]
0    0
1    1
2    1
3    1
4    0
dtype: int64
-----------------
[0, 0, 1, 0, 0, 1]
0    0
1    1
2    0
3    0
4    0
dtype: int64
-----------------
[0, 0, 1, 0, 0, 1]
0    0
1    0
2    1
3    1
4    0
dtype: int64
-----------------
[1, 1, 2, 0, 0, 1]
0    1
1    1
2    1
3    1
4    0
dtype: int64
-----------------
[0, 1, 1, 0, 1, 1]
0    1
1    1
2    0
3    1
4    0
dtype: int64
-----------------
[0, 1, 0, 0, 1, 0]
0    1
1    0
2    0
3    1
4    0
dtype: int64
-----------------


[0    0
 1    1
 2    1
 3    1
 4    0
 dtype: int64,
 0    0
 1    1
 2    0
 3    0
 4    0
 dtype: int64,
 0    0
 1    0
 2    1
 3    1
 4    0
 dtype: int64,
 0    1
 1    1
 2    1
 3    1
 4    0
 dtype: int64,
 0    1
 1    1
 2    0
 3    1
 4    0
 dtype: int64,
 0    1
 1    0
 2    0
 3    1
 4    0
 dtype: int64]

In [259]:
check_decide_y_fuc_mul(X_ori55_ad, ori_y5, selected_col_num2, deci_y1)

In [260]:
dot_list_confirm1 = [np.dot(X_ori55_ad[i], deci_y1[0]) - np.dot(X_ori55_ad[j], deci_y1[0]) for i in select_cols for j in else_cols]
dot_list_confirm1

[0, 0, 2, 0, 0, 2]

In [263]:
# 出力一つ、yに含まれる1を固定
def decide_y_fix_y1(X, ori_y, infer_y, selected_col_num, y1_sum):
    select_cols, else_cols = featrure_InnerProduct_sel_mul(X, ori_y, selected_col_num)
    deci_y = []
    dot_list = [np.dot(X[i], infer_y) - np.dot(X[j], infer_y) for i in select_cols for j in else_cols]
    
    if len([ll for ll in dot_list if ll <0]) == 0 and sum(infer_y) == y1_sum:
        deci_y.append(copy.deepcopy(infer_y))
    else:
        for kk in reversed(range(num_samples)):
            for k in reversed(range(kk+1)):
                infer_y[k] = 1 - infer_y[k]
                dot_list = [np.dot(X[i], infer_y) - np.dot(X[j], infer_y) for i in select_cols for j in else_cols]
                if len([ll for ll in dot_list if ll <0]) == 0 and sum(infer_y) == y1_sum:
                    deci_y.append(copy.deepcopy(infer_y))

                else:
                    infer_y[k] = 1 - infer_y[k]
            infer_y[kk] = 1 - infer_y[kk]
    return deci_y

In [264]:
infer_y0 = pd.Series([0, 1, 1, 0, 1])
deci_y2 = decide_y_fix_y1(X_ori55_ad, ori_y5, infer_y0, selected_col_num2, y1_sum)
deci_y2#なし

[0    0
 1    1
 2    1
 3    1
 4    0
 dtype: int64]

In [273]:
# 出力複数、yに含まれる1を固定
def decide_y_fix_y1_mul(X, ori_y, infer_y, selected_col_num, y1_sum):
    select_cols, else_cols = featrure_InnerProduct_sel_mul(X, ori_y, selected_col_num)
    deci_y_list = []
    dot_list = [np.dot(X[i], infer_y) - np.dot(X[j], infer_y) for i in select_cols for j in else_cols]
    
    if len([ll for ll in dot_list if ll <0]) == 0 and sum(infer_y) == y1_sum:
        deci_y_list.append(copy.deepcopy(infer_y))
        
    for kk in reversed(range(num_samples)):
        for k in reversed(range(kk+1)):
            infer_y[k] = 1 - infer_y[k]
            dot_list = [np.dot(X[i], infer_y) - np.dot(X[j], infer_y) for i in select_cols for j in else_cols]
            if len([ll for ll in dot_list if ll <0]) == 0 and sum(infer_y) == y1_sum:
                if (len(deci_y_list) == 0) or (deci_y_list[-1] == infer_y).all() == False:
                    deci_y_list.append(copy.deepcopy(infer_y))
            infer_y[k] = 1 - infer_y[k]
        infer_y[kk] = 1 - infer_y[kk]
        
    return deci_y_list

In [277]:
infer_y0 = pd.Series([0, 1, 1, 0, 1])
deci_y3 = decide_y_fix_y1_mul(X_ori55_ad, ori_y5, infer_y0, selected_col_num2, y1_sum)
deci_y3

[0    0
 1    1
 2    1
 3    1
 4    0
 dtype: int64,
 0    1
 1    1
 2    0
 3    1
 4    0
 dtype: int64]

In [278]:
check_decide_y_fuc_mul(X_ori55_ad, ori_y5, selected_col_num2, deci_y3)

In [279]:
dot_list_confirm3 = [np.dot(X_ori55_ad[i], deci_y3[0]) - np.dot(X_ori55_ad[j], deci_y3[0]) for i in select_cols for j in else_cols]
dot_list_confirm3

[0, 0, 2, 0, 0, 2]